# Classificador de Fake News e proposta para não-proliferação das mesmas

   O assunto de Fake News está em alta na mídia já a um bom tempo, e teve seu pico quando o presidente dos Estados Unidos, Donald Trump, com a sua conta de Twitter, começou a disseminar a palavra sobre supostos sites de notícias que lançavam "notícias falsas". Tendo em mente isso, visamos a criar um programa utilizando de Machine Learning que faz o trabalho de decidir, utilizando a base de Natural Language Processing (NLP), se uma notícia realmente é falsa ou não. Para isso, este Notebook estará dividido nas seguintes partes: 
  - Pré-Processamento
  - Análise Exploratória dos Dados
  - Aplicação do Modelo de Machine Learning 
  - Validação e conclusão do Modelo

## Importação de libraries e de dados

In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
import simplejson
import re
from tqdm import tqdm 
from goose3 import Goose

In [2]:
# Importamos uma certa quantia de dados do nosso arquivo json.

data_raw = []
num_data = 5000
with open('data/fake-news-tera.json') as file:
    for i in tqdm(range(num_data)):
        line = next(file)
        data_raw.append(simplejson.loads(line))
        
df_raw = pd.DataFrame(data_raw)

100%|██████████| 5000/5000 [00:00<00:00, 5091.78it/s]


## Pré-processamento

In [3]:
# Primeira olhada nos nossos dados

df_raw.head()

,app,content,date,location_city,location_country,location_lat,location_lng,location_state,profile_avatar,profile_description,...,profile_location,profile_login,profile_name,profile_url,profile_web,related_tweet_content,related_tweet_date,related_tweet_url,source,url
0,Facebook,Veja o passo a passo: http://fb.me/CK6knq9W,2017-10-31T06:00:03.000Z,Brás de Pina,Brasil,-22.832979,-43.297668,Rio de Janeiro,https://pbs.twimg.com/profile_images/339749594...,"Em breve, o melhor GUIA TURÍSTICO do Rio | Soo...",...,Rio de Janeiro (BR),amigonorio,Amigo no Rio,https://twitter.com/amigonorio,http://www.amigonorio.com.br,None,None,None,0,https://twitter.com/608352107/statuses/9252409...
1,Twitter for iPhone,@maika_1020 BAND MAIDのchoose me(｡>人<),2017-11-01T05:59:13.000Z,None,None,NaN,NaN,None,https://pbs.twimg.com/profile_images/920977346...,MOSHIMO SILENT SIREN sevenoops,...,None,msyn_711,まっしゃん,https://twitter.com/msyn_711,None,None,None,None,0,https://twitter.com/621782721/statuses/9256031...
2,Twitter Web Client,"É DONO DO STF. ONDE TODOS SÃO COVARDES, CONFOR...",2017-11-01T06:00:04.000Z,None,Brasil,-10.000000,-55.000000,None,https://pbs.twimg.com/profile_images/919944565...,"QUANDO OS QUE COMANDAM PERDEM A VERGONHA,OS CO...",...,Brasil,CarlitoMoraes,Carlito Moraes,https://twitter.com/CarlitoMoraes,None,Absurdo ... Gilmar se sente o dono do Brasil ....,2017-10-31T14:19:45.000Z,https://twitter.com/887707008/statuses/9253666...,0,https://twitter.com/346220548/statuses/9256033...
3,Twitter Lite,porra a globo corta se fodrrrr,2017-11-03T05:59:52.000Z,Santos,Brasil,-23.856630,-46.270550,São Paulo,https://pbs.twimg.com/profile_images/925930977...,musa de Alceu Valença•\n@justinbieber💙,...,baixada santista,ingcarvalh0,dindi,https://twitter.com/ingcarvalh0,https://www.instagram.com/morena7ropicana/?hl=...,None,None,None,0,https://twitter.com/1315339214/statuses/926328...
4,Twitter Lite,@UOL Deviam ter deixado mata-lo,2017-11-05T05:58:10.000Z,None,None,NaN,NaN,None,https://pbs.twimg.com/profile_images/905322029...,enfrentando novos desafios e vivendo um dia de...,...,None,wall2013,wall2013,https://twitter.com/wall2013,http://walterdesouza2012.blogspot.com.br/,None,None,None,0,https://twitter.com/1172898380/statuses/927052...


In [4]:
# Para nossa análise, foi concluído que não precisamos de algumas das colunas de nosso DataFrame, então foram tiradas:

df_clean = df_raw.drop(['app', 'location_city', 'location_country', 'location_lat', 'location_lng', 'location_state', 'profile_avatar', 'profile_location', 'profile_name', 'profile_web', 'related_tweet_date'], axis=1)
df_clean.head()

,content,date,profile_description,profile_followers,profile_following,profile_login,profile_url,related_tweet_content,related_tweet_url,source,url
0,Veja o passo a passo: http://fb.me/CK6knq9W,2017-10-31T06:00:03.000Z,"Em breve, o melhor GUIA TURÍSTICO do Rio | Soo...",18.0,15.0,amigonorio,https://twitter.com/amigonorio,None,None,0,https://twitter.com/608352107/statuses/9252409...
1,@maika_1020 BAND MAIDのchoose me(｡>人<),2017-11-01T05:59:13.000Z,MOSHIMO SILENT SIREN sevenoops,695.0,674.0,msyn_711,https://twitter.com/msyn_711,None,None,0,https://twitter.com/621782721/statuses/9256031...
2,"É DONO DO STF. ONDE TODOS SÃO COVARDES, CONFOR...",2017-11-01T06:00:04.000Z,"QUANDO OS QUE COMANDAM PERDEM A VERGONHA,OS CO...",5694.0,4225.0,CarlitoMoraes,https://twitter.com/CarlitoMoraes,Absurdo ... Gilmar se sente o dono do Brasil ....,https://twitter.com/887707008/statuses/9253666...,0,https://twitter.com/346220548/statuses/9256033...
3,porra a globo corta se fodrrrr,2017-11-03T05:59:52.000Z,musa de Alceu Valença•\n@justinbieber💙,2641.0,1098.0,ingcarvalh0,https://twitter.com/ingcarvalh0,None,None,0,https://twitter.com/1315339214/statuses/926328...
4,@UOL Deviam ter deixado mata-lo,2017-11-05T05:58:10.000Z,enfrentando novos desafios e vivendo um dia de...,159.0,953.0,wall2013,https://twitter.com/wall2013,None,None,0,https://twitter.com/1172898380/statuses/927052...


In [5]:
# Vemos o shape de nossos dados e suas características

print('Shape:\n')
print(df_clean.shape)
print(10*'-')
print('Contagem de NaN:\n')
print(df_clean.isna().sum())
print(10*'-')
print('Contagem de Null:\n')
print(df_clean.isnull().sum())
print(10*'-')

Shape:

(5000, 11)
----------
Contagem de NaN:

content                     0
date                        0
profile_description      1089
profile_followers          89
profile_following          89
profile_login               0
profile_url                 0
related_tweet_content    4571
related_tweet_url        4571
source                      0
url                         0
dtype: int64
----------
Contagem de Null:

content                     0
date                        0
profile_description      1089
profile_followers          89
profile_following          89
profile_login               0
profile_url                 0
related_tweet_content    4571
related_tweet_url        4571
source                      0
url                         0
dtype: int64
----------


Quando analisamos os dados acima, podemos ver que aproximadamente 90% dos dados não tem 'related_tweet_content', porém não é bom tira-los do nosso dataset, pois é um dos lugares aonde é possível puxar a URL da notícia que queremos.

### !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! 

In [6]:
# Para que possamos analisar as notícias mencionadas nos tweets em nosso dataset, precisamos da URL da mesma
# Assim, criamos uma função para receber uma STRING e retornar o link contido na mesma. Caso não haja, retorna None

def find_url(raw):
    if raw == None:
        return None
    txt = re.search('(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', raw)
    if txt:
        return txt.group()
    else:
        return None

In [7]:
# E então, analisando os dados, vimos que o link para a notícia pode estar tanto na coluna 'content' quanto na de
# 'related_tweet_content', portanto para criar a coluna 'URL', contento as URL que posteriormente tem que ser ana-
# lisadas, checamos primeiramente se há na 'related_tweet_content', depois no 'content', se existirem

df_clean['URL'] = None # Coluna de URL, que será atualizada 
for i in tqdm(range(num_data)):
    cont = find_url(df_clean['content'][i])
    re_cont = find_url(df_clean['related_tweet_content'][i])
    if find_url(str(re_cont)) != None:
        df_clean.loc[i, 'URL'] = re_cont
    elif cont != None:
        df_clean.loc[i, 'URL'] = cont
    else:
        df_clean.loc[i, 'URL'] = None

df_clean['URL'].head() # Verificação do output

100%|██████████| 5000/5000 [00:03<00:00, 1271.13it/s]


0       http://fb.me/CK6knq9W
1                        None
2    http://ow.ly/boor30gfBli
3                        None
4                        None
Name: URL, dtype: object

In [ ]:
# Agora, de cada URL precisamos pegar o corpo do texto. Para nossa sorte, a biblioteca goose3 tem um módulo que 
# faz exatamente isso para nós. :) 
# Note que esse processo é demorado. 

df_clean['Article'] = None
g = Goose()
for i in tqdm(range(num_data)):
    url = df_clean.loc[i, 'URL']
    if not(url):
        df_clean.loc[i, 'Article'] = None
    else:
        try:
            article = g.extract(url=url)
            df_clean.loc[i, 'Article'] = article.cleaned_text
        except:
            df_clean.loc[i, 'Article'] = None

  7%|▋         | 369/5000 [03:38<45:41,  1.69it/s]File not found
Traceback (most recent call last):
  File "/home/marcello/anaconda3/lib/python3.6/site-packages/goose3/utils/images.py", line 44, in get_image_dimensions
    with Image.open(img_file) as image:
  File "/home/marcello/anaconda3/lib/python3.6/site-packages/PIL/Image.py", line 2319, in open
    % (filename if filename else fp))
OSError: cannot identify image file <_io.BufferedReader name='/tmp/goose/9c750a55e03b665c570d63f1a554c6e4.1512873594.0397499_193e86f76a27012d6904aaacc8290000'>
File not found
Traceback (most recent call last):
  File "/home/marcello/anaconda3/lib/python3.6/site-packages/goose3/utils/images.py", line 44, in get_image_dimensions
    with Image.open(img_file) as image:
  File "/home/marcello/anaconda3/lib/python3.6/site-packages/PIL/Image.py", line 2319, in open
    % (filename if filename else fp))
OSError: cannot identify image file <_io.BufferedReader name='/tmp/goose/9c750a55e03b665c570d63f1a554c6e4.